# 랭체인의 구성 요소

## 모델과 프롬프트

In [19]:
from langchain_openai import OpenAI
import os

os.environ["OPENAI_API_KEY"][:3]  # 앞부분만 출력

'sk-'

In [23]:
llm = OpenAI()
print(llm.invoke('농담을 해봐.'))


"왜 닭은 도로를 건너야 하나요? 물론, 계란을 횡단보도에 놓고 건너기 때문이죠!"


In [24]:
from langchain import PromptTemplate

template = """문장: {sentence}
{language}로 번역:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

print(prompt.format(sentence = "탁자 위에 고양이가 있다", language = "영어"))

문장: 탁자 위에 고양이가 있다
영어로 번역:


## 데이터 연결

### 문서 로더

In [10]:
import csv

# Sample data
data = [
    ['이름', '나이', '도시'],
    ['존', 25, '뉴욕'],
    ['에밀리', 28, '로스엔젤레스'],
    ['미카엘', 22, '시카고']
]

# File name
file_name = 'sample.csv'

# 데이터를 CSV 파일에 기록
with open(file_name, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

print(f'예제 CSV 파일 "{file_name}"를 만들었습니다.')



예제 CSV 파일 "sample.csv"를 만들었습니다.


In [9]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='sample.csv')
data = loader.load()
print(data)

[Document(page_content='이름: 존\n나이: 25\n도시: 뉴욕', metadata={'source': 'sample.csv', 'row': 0}), Document(page_content='이름: 에밀리\n나이: 28\n도시: 로스엔젤레스', metadata={'source': 'sample.csv', 'row': 1}), Document(page_content='이름: 미카엘\n나이: 22\n도시: 시카고', metadata={'source': 'sample.csv', 'row': 2})]


### 문서 분할기

In [11]:
# 산과 자연에 대한 샘플 문장
content = """고요한 풍경 속에서 우뚝 솟은 산들은 자연의 아름다움을 지키는 장엄한 수호자처럼 서 있습니다.
청량한 산 공기는 고요함의 속삭임을 전해주며, 바스락거리는 잎사귀들은 야생의 교향곡을 작곡합니다.
자연의 팔레트는 산을 초록과 갈색의 색조로 칠해 경이로운 광경을 만들어냅니다.
해가 뜨면, 황금빛 광채가 산 정상에 비치며, 손길 닿지 않은 야생의 세계를 비춥니다."""

# 파일명
file_name = 'mountain.txt'

# 텍스트 파일에 내용 쓰기
with open(file_name, 'w') as txtfile:
    txtfile.write(content)

# 샘플 텍스트 파일 "mountain.txt" 생성 및 저장.
# print(f'샘플 텍스트 파일 "{file_name}"를 만들었습니다.')

with open('mountain.txt') as f:
    mountain = f.read()

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len
)

texts = text_splitter.create_documents([mountain])
print(texts[0])
print(texts[1])
print(texts[2])


page_content='고요한 풍경 속에서 우뚝 솟은 산들은 자연의 아름다움을 지키는 장엄한 수호자처럼 서 있습니다.'
page_content='청량한 산 공기는 고요함의 속삭임을 전해주며, 바스락거리는 잎사귀들은 야생의 교향곡을 작곡합니다.\n자연의 팔레트는 산을 초록과 갈색의 색조로 칠해 경이로운 광경을 만들어냅니다.'
page_content='해가 뜨면, 황금빛 광채가 산 정상에 비치며, 손길 닿지 않은 야생의 세계를 비춥니다.'


### 텍스트 임베딩 모델

In [31]:
from langchain_openai import OpenAIEmbeddings

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]

embeddings_model = OpenAIEmbeddings(model ='text-embedding-3-small' )

embeddings = embeddings_model.embed_documents(
    [
        "Good morning!",
        "Oh, hello!",
        "I want to report an accident",
        "Sorry to hear that. May I ask your name?",
        "Sure, Mario Rossi."
    ]
)

print("임베드된 문서:")
print(f"Number of vector: {len(embeddings)}; Dimension of each vector: {len(embeddings[0])}")

embedded_query = embeddings_model.embed_query("What was the name mentioned in the conversation?")

print("임베드 질의:")
print(f"Dimension of the vector: {len(embedded_query)}")
print(f"Sample of the first 5 elements of the vector: {embedded_query[:5]}")


임베드된 문서:
Number of vector: 5; Dimension of each vector: 1536
임베드 질의:
Dimension of the vector: 1536
Sample of the first 5 elements of the vector: [-0.010634176433086395, -0.01016946416348219, -0.0020040736999362707, 0.023065242916345596, -0.026829415932297707]


In [34]:
# 대화를 txt 파일에 저장
# 대화 행 목록
dialogue_lines = [
    "Good morning!",
    "Oh, hello!",
    "I want to report an accident",
    "Sorry to hear that. May I ask your name?",
    "Sure, Mario Rossi."
]

# 파일명
file_name = 'dialogue.txt'

# 대화 행들을 텍스트 파일에 기록
with open(file_name, 'w') as txtfile:
    for line in dialogue_lines:
        txtfile.write(line + '\n')

print(f'대화 텍스트 파일 "{file_name}"를 만들었습니다.')


대화 텍스트 파일 "dialogue.txt"를 만들었습니다.


### 벡터 스토어

In [35]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"]

# 문서를 로드해 청크들로 분할하고, 각 청크를 임베드해 벡터 스토어에 로드

raw_documents = TextLoader('dialogue.txt').load()
text_splitter = CharacterTextSplitter(chunk_size=50, chunk_overlap=0, separator = "\n",)
documents = text_splitter.split_documents(raw_documents)
db = FAISS.from_documents(documents, OpenAIEmbeddings())


In [36]:
query = "What is the reason for calling?"
docs = db.similarity_search(query)
print(docs[0].page_content)

I want to report an accident


In [37]:
print(documents[2])

page_content='Sorry to hear that. May I ask your name?' metadata={'source': 'dialogue.txt'}


### 검색기

In [38]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

retriever = db.as_retriever()

In [42]:
qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever)

query = "What was the reason of the call?"
output = qa.invoke(query)
output['result']

' The reason for the call was to report an accident.'

## 메모리

In [43]:
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.llms import OpenAI

memory = ConversationSummaryMemory(llm=OpenAI(temperature=0))
memory.save_context({"input": "안녕하세요, AI에 관한 에세이를 쓸 아이디어를 찾고 있어요"}, {"output": "안녕하세요, LLM에 관해 써보면 어떨까요?"})

memory.load_memory_variables({})

{'history': '\nThe human is looking for ideas to write an essay about AI. The AI suggests writing about LLM.'}

In [44]:
ConversationSummaryMemory.save_context?

Signature:
ConversationSummaryMemory.save_context(
    self,
    inputs: 'Dict[str, Any]',
    outputs: 'Dict[str, str]',
) -> 'None'
Docstring: Save context from this conversation to buffer.
File:      c:\users\yong\appdata\local\programs\python\python311\lib\site-packages\langchain\memory\summary.py
Type:      function

## 체인

### 단순 체인

In [45]:
from langchain import PromptTemplate, OpenAI, LLMChain

template = """문장: {sentence}
{language}로 번역:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

llm = OpenAI(temperature=0)

llm_chain = LLMChain(prompt=prompt, llm=llm)

llm_chain.predict(sentence="탁자 위에 고양이가 있어요", language="영어")

C:\Users\yong\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


' There is a cat on the table.'

In [48]:
# LLMChain deprecation 해결

from langchain import PromptTemplate, OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence, RunnablePassthrough

template = """문장: {sentence}
{language}로 번역:"""
prompt = PromptTemplate(template=template, input_variables=["sentence", "language"])

llm = OpenAI(temperature=0)

output_parser = StrOutputParser()

chain = RunnableSequence(
    {
        "sentence": RunnablePassthrough(),
        "language": RunnablePassthrough()
    }
    | prompt
    | llm
    | output_parser
)

result = chain.invoke({
    "sentence": "탁자 위에 고양이가 있어요",
    "language": "영어"
})
print(result)



There is a cat on the table.


### 라우터 체인

In [51]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

llm = OpenAI()

itinerary_template = """당신은 휴가 일정 조수입니다. \
당신은 고객이 최고의 목적지와 일정을 찾도록 도와줍니다. \
당신은 고객의 선호에 따라 최적화된 일정을 작성하는 데 도움을 줍니다.

여기에 질문이 있습니다:
{input}"""

restaurant_template = """당신은 레스토랑 예약 조수입니다. \
당신은 고객의 손님 수와 음식 선호를 확인합니다. \
특별한 조건을 고려해야 하는지 주의합니다.

여기에 질문이 있습니다:
{input}"""

prompt_infos = [
    {
        "name": "여행 일정",
        "description": "여행 일정 작성을 돕습니다",
        "prompt_template": itinerary_template,
    },
    {
        "name": "레스토랑",
        "description": "고객의 레스토랑 예약을 도와줍니다",
        "prompt_template": restaurant_template,
    },
]

destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain
default_chain = ConversationChain(llm=llm, output_key="text")

destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)



In [56]:
print(chain.run("밀라노에서 베니스까지 자동차로 여행하려고 합니다. 중간에 들릴 만한 명소가 있나요?."))



> Entering new MultiPromptChain chain...
여행 일정: {'input': '밀라노에서 베니스까지 자동차로 여행하려고 합니다. 중간에 들릴 만한 명소가 있나요?.'}
> Finished chain.


1. 베니스에 가기 전에 들르기 좋은 도시는 어디인가요?
2. 베니스에서 머물만한 좋은 호텔은 어디인가요?
3. 베니스에서 즐길 수 있는 가장 인기있는 여행지는 어디인가요?
4. 베니스에서 가장 유명한 레스토랑이 어디인가요?
5. 베니스에서 가장 인기있는 액티비티는 무엇인가요?
6. 베니스에서 볼 수 있는 유명한 건축물은 어디인가요?
7. 베니스에서 가장 아름다운 풍경을 볼 수 있는 곳은 어디인가요?
8. 베니스에서 쇼핑하기 좋은 곳은 어디인가요?
9. 베니스에서 가장 유명


In [57]:
print(chain.run("오늘 저녁 식사를 예약하고 싶어요"))



> Entering new MultiPromptChain chain...
레스토랑: {'input': '오늘 저녁 식사를 예약하고 싶어요'}
> Finished chain.
. 몇 명이서 식사하시겠어요?
어떤 종류의 음식을 선호하시나요?
특별한 요구사항이 있나요? (알레르기, 식이 요구사항 등)


### 시퀀셜 체인

In [58]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# This is an LLMChain to write a synopsis given a title of a play.
llm = OpenAI(temperature=0)
template = """당신은 코미디언입니다. {topic}에 관한 농담을 생성하세요.
농담:"""
prompt_template = PromptTemplate(input_variables=["topic"], template=template)
joke_chain = LLMChain(llm=llm, prompt=prompt_template)

template = """당신은 번역가입니다. 주어진 텍스트 입력을 {language}로 번역하세요.
번역:"""
prompt_template = PromptTemplate(input_variables=["language"], template=template)
translator_chain = LLMChain(llm=llm, prompt=prompt_template)

In [59]:
# This is the overall chain where we run these two chains in sequence.
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[joke_chain, translator_chain], verbose=True)
translated_joke = overall_chain.run("고양이와 개")



> Entering new SimpleSequentialChain chain...
 "고양이와 개가 싸우면 누가 이길까요? 당연히 고양이죠, 개는 꼬리를 쫓는데 바쁘니까!"
 If cats and dogs fight, who do you think will win? Of course, it's the cat, because the dog is too busy chasing its tail!

> Finished chain.


### 변환 체인

In [60]:
# string 모듈을 임포트합니다
import string

# 함수를 정의합니다
def rename_cat(inputs: dict) -> dict:
    # 파일을 읽기 모드로 엽니다
    text = inputs["text"]
    # 'cat'을 'Silvester the Cat'으로 바꿉니다
    new_text = text.replace('cat', 'Silvester the Cat')
    # 변경된 텍스트를 반환합니다
    return {"output_text": new_text}


In [61]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

with open("Cats&Dogs.txt") as f:
    cats_and_dogs = f.read()


import string



transform_chain = TransformChain(
    input_variables=["text"], output_variables=["output_text"], transform=rename_cat
)

template = """이 텍스트를 요약하세요:

{output_text}

요약:"""
prompt = PromptTemplate(input_variables=["output_text"], template=template)
llm_chain = LLMChain(llm=OpenAI(), prompt=prompt)

sequential_chain = SimpleSequentialChain(chains=[transform_chain, llm_chain])

sequential_chain.run(cats_and_dogs)

' 실베스터와 개가 같은 집에서 살았는데 실베스터는 장난을 잘 치고, 개는 충실하고 친근했다. 어느 날 실베스터는 개를 속이기 위해 실을 발을 붙여놓고 숨었다. 개는 이를 알아채고 실베스터를 쫓다가 싸웠지만 주인에게 혼나고 다시 친구가 되었다.'

## 에이전트

In [62]:
from langchain import SerpAPIWrapper
from langchain.agents import AgentType, initialize_agent
from langchain.llms import OpenAI
from langchain.tools import BaseTool, StructuredTool, Tool, tool

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["SERPAPI_API_KEY"]

search = SerpAPIWrapper()

In [63]:
tools = [Tool.from_function(
        func=search.run,
        name="Search",
        description="현재 이벤트에 관해 질문할 때 유용함"
    )]

agent = initialize_agent(tools, llm = OpenAI(), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

agent.run("아바타 2 개봉일은?")

C:\Users\yong\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(




> Entering new AgentExecutor chain...
 아바타 2는 언제쯤 개봉할까 생각해보자
Action:Search
Action Input: "Avatar 2 release date"
Observation: December 16, 2022
Thought: 그렇구나, 아바타 2는 2022년 12월 16일에 개봉한다는 거구나!
Final Answer: December 16, 2022

> Finished chain.


'December 16, 2022'

# 허깅페이스 허브를 통해 LLM 사용하기

In [10]:
#필요한 패키지 설치
!pip install python-dotenv   
!pip install langchain-huggingface


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/43.7 kB ? eta -:--:--
     ---------------------------------------- 43.7/43.7 kB 2.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/417.5 kB ? eta -:--:--
   ---------------------------------------- 417.5/417.5 kB 8.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/227.1 kB ? eta -:--:--
   ---------------------------------------- 227.1/227.1 kB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------- ----------------------- 0.9/2.2 MB 19.2 MB/s eta 0:00:01
   ------------------------------------ --- 2.0/2.2 MB 21.5 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 20.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
   ---- ----------------------------------- 1.1/9.5 MB 36.0 MB/s eta 0:00:01
   ---------- ----------------------------- 2.6/9.5 MB 33.1 MB/s eta 0:00:01
   ---------------- ---


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
#옵션 1: .env 파일에서 토큰 가져기기

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["HUGGINGFACEHUB_API_TOKEN"][:3]  # 앞부분만 출력


'hf_'

In [1]:
#옵션 2: getpass 함수를 사용해 토큰 가져오기

# from getpass import getpass

# HUGGINGFACEHUB_API_TOKEN = getpass()
# HUGGINGFACEHUB_API_TOKEN

In [11]:
from langchain import PromptTemplate, LLMChain
from langchain_huggingface import HuggingFaceEndpoint
question = "What was the first Disney movie?"

template = """Question: {question}

Answer: give a direct answer"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [13]:
repo_id = "tiiuae/falcon-7b-instruct"  
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    max_length=1000,
    temperature=0.5,
)
print(llm("what was the first disney movie?"))

WARNING! max_length is not default parameter.
                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\yong\.cache\huggingface\token
Login successful

The first Disney movie was 'Snow White and the Seven Dwarfs', released in 1937.
